# K-MCI
### Intro
In this notebook we have the main algorithm of the article. All necessary functions are implemented in function.py file.<br>
This document have blow sections:<br>
1. Importing Libraries & classes
2. Reading datasets
3. Preprocessing datasets
4. 

### Importing Libraries & classes
We import some libraries from third party and our impelmented .py files.<br>
there are three .py files described blow:
1. <span><em><strong>candidate_class.py</strong></em></span>: Contains the implementation of Candidate object according to the article(nodes container!).
2. <em><strong>datasets.py</strong></em>: In this class we read our <strong>.csv</strong> datasets using Pandas dataframes and do some preprocessing like handling missing values, encoding variables, separating features from target values and ... .
3. <em><strong>functions.py</strong></em>: We developed some functions which are key part of our implementation and put them all into this class to prevent audiance from unnecessary detailed information.

### Importing Libraries & classes

In [9]:
import datasets
import candidate_class as cn
#import functions

In [10]:
Candidate_1 = cn.Candidate(1,5,15,-1)
Candidate_1.cluster

AttributeError: 'Candidate' object has no attribute 'centers'